In [ ]:
#SPAM email analysis using BERT

In [5]:
import tensorflow_hub as hub
import tensorflow_text as text

In [7]:
preprocessor = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encode = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [8]:
bert_preprocessed_model = hub.KerasLayer(preprocessor)

In [11]:
test_text = ["I am a good boy"]
test_text_pre = bert_preprocessed_model(test_text)
test_text_pre.keys()

dict_keys(['input_mask', 'input_word_ids', 'input_type_ids'])

In [16]:
bert_model = hub.KerasLayer(encode)
bert_result = bert_model(test_text_pre)
bert_result.keys()


dict_keys(['default', 'encoder_outputs', 'pooled_output', 'sequence_output'])

In [17]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf

import pandas as pd

In [19]:
df = pd.read_csv(r"C:\Users\HP\Downloads\SPAM text message 20170820 - Data.csv")

In [20]:
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [21]:
df.groupby("Category").describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [24]:
df_spam = df[df["Category"]=="spam"]
df_spam.shape

(747, 2)

In [25]:
df_ham = df[df["Category"]=="ham"]
df_ham.shape

(4825, 2)

In [26]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [28]:
df_balanced = pd.concat([df_spam,df_ham_downsampled])

In [31]:
df_balanced.describe()

,Category,Message
count,1494,1494
unique,2,1375
top,spam,"Sorry, I'll call later"
freq,747,4


In [33]:
df_balanced["spam"] = df_balanced["Category"].apply(lambda x: 1 if x =="spam" else 0)

In [34]:
df_balanced

,Category,Message,spam
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",1
...,...,...,...
581,ham,Huh so early.. Then ü having dinner outside iz...,0
4031,ham,"Cool, I'll text you in a few",0
3974,ham,"Night night, see you tomorrow",0
3960,ham,Have a nice day my dear.,0


In [40]:
from sklearn.model_selection import train_test_split

In [77]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [43]:
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encode = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [54]:
def get_sentance_embedding(sentance):
    
    preprocessed_text = preprocessor(sentance)
    return encode(preprocessed_text)["pooled_output"]
   

get_sentance_embedding([
    "Hurry up, Sales are going on",
    "It's 50 percent off"
])



<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.906983  , -0.51244235, -0.957087  , ..., -0.7844184 ,
        -0.766721  ,  0.9248904 ],
       [-0.8260253 , -0.16887715,  0.37223634, ...,  0.2827168 ,
        -0.56309116,  0.8345314 ]], dtype=float32)>

In [58]:
e = get_sentance_embedding(["banana","grapes","mangoes","jeff bezoz","elon musk"])

In [73]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessor(text_input)
outputs = encode(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [63]:
text_input

<KerasTensor: shape=(None,) dtype=string (created by layer 'text')>

In [74]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [75]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [78]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 291s 8s/step - loss: 0.6597 - accuracy: 0.5875 - precision: 0.5942 - recall: 0.5518
Epoch 2/10
35/35 [==============================] - 274s 8s/step - loss: 0.5296 - accuracy: 0.7946 - precision: 0.7750 - recall: 0.8304
Epoch 3/10
35/35 [==============================] - 273s 8s/step - loss: 0.4657 - accuracy: 0.8348 - precision: 0.8295 - recall: 0.8429
Epoch 4/10
35/35 [==============================] - 265s 8s/step - loss: 0.4119 - accuracy: 0.8643 - precision: 0.8458 - recall: 0.8911
Epoch 5/10
35/35 [==============================] - 326s 9s/step - loss: 0.3809 - accuracy: 0.8696 - precision: 0.8520 - recall: 0.8946
Epoch 6/10
35/35 [==============================] - 346s 10s/step - loss: 0.3521 - accuracy: 0.8938 - precision: 0.8808 - recall: 0.9107
Epoch 7/10
35/35 [==============================] - 370s 11s/step - loss: 0.3323 - accuracy: 0.8875 - precision: 0.8794 - recall: 0.8982
Epoch 8/10
35/35 [============================

In [80]:
y_predeict = model.predict(X_test)
#y_predeict = y_predeict.flattern()

12/12 [==============================] - 97s 8s/step


In [83]:
import numpy as np
y_predeict = np.where(y_predeict > 0.5,1,0)

In [84]:
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(y_test,y_predeict)

In [85]:
cm

array([[158,  29],
       [  7, 180]], dtype=int64)

In [88]:
print(classification_report(y_test,y_predeict))

              precision    recall  f1-score   support

           0       0.96      0.84      0.90       187
           1       0.86      0.96      0.91       187

    accuracy                           0.90       374
   macro avg       0.91      0.90      0.90       374
weighted avg       0.91      0.90      0.90       374

